In [1]:
from imports import *

# set columnns display format
pd.set_option('display.max_columns', None)
# default pandas decimal number display format
pd.options.display.float_format = '{:20,.3f}'.format

# Data Acquisition

In [2]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
def new_zillow_data():
    '''
    This function reads the Zillow data from the mySQL database into a df.
    '''
    # Create SQL query.
    sql_query = '''
    SELECT 
    prop.*,
    pred.logerror,
    pred.transactiondate,
    air.airconditioningdesc,
    arch.architecturalstyledesc,
    build.buildingclassdesc,
    heat.heatingorsystemdesc,
    landuse.propertylandusedesc,
    story.storydesc,
    construct.typeconstructiondesc
FROM
    properties_2017 prop
        INNER JOIN
    (SELECT 
        parcelid, logerror, MAX(transactiondate) AS transactiondate
    FROM
        predictions_2017
    GROUP BY parcelid ,  logerror) pred USING (parcelid)
        LEFT JOIN
    airconditioningtype air USING (airconditioningtypeid)
        LEFT JOIN
    architecturalstyletype arch USING (architecturalstyletypeid)
        LEFT JOIN
    buildingclasstype build USING (buildingclasstypeid)
        LEFT JOIN
    heatingorsystemtype heat USING (heatingorsystemtypeid)
        LEFT JOIN
    propertylandusetype landuse USING (propertylandusetypeid)
        LEFT JOIN
    storytype story USING (storytypeid)
        LEFT JOIN
    typeconstructiontype construct USING (typeconstructiontypeid)
WHERE prop.propertylandusetypeid = 261 AND
        prop.latitude IS NOT NULL
        AND prop.longitude IS NOT NULL
        AND transactiondate <= '2017-12-31';
    '''
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    df = df.drop(columns='id')
    return df

def get_zillow_data():
    '''
    This function reads in zillow data from Zillow database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('zillow_data.csv'):
        
        # If csv file exists, read in data from csv file.
        df = pd.read_csv('zillow_data.csv', index_col=0)
    else:
        
        # Read fresh data from db into a DataFrame.
        df = new_zillow_data()
        
        # Write DataFrame to a csv file.
        df.to_csv('zillow_data.csv')
        
    return df

In [4]:
df = get_zillow_data()

In [5]:
df.shape

(52438, 67)

# Data Preparation

### 1. Missing Values

In [6]:
def handle_missing_values(df, prop_required_columns, prop_required_row):
    threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold) #1, or ‘columns’ : Drop columns which contain missing value
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold) #0, or ‘index’ : Drop rows which contain missing values.
    return df

In [7]:
df = handle_missing_values(df, 0.75, 0.75)

In [8]:
df.shape

(52434, 28)

In [9]:
df.isnull().sum()

parcelid                           0
bathroomcnt                        0
bedroomcnt                         0
calculatedbathnbr                133
calculatedfinishedsquarefeet      78
finishedsquarefeet12             243
fips                               0
fullbathcnt                      133
latitude                           0
longitude                          0
lotsizesquarefeet                366
propertycountylandusecode          0
propertylandusetypeid              0
rawcensustractandblock             0
regionidcity                    1033
regionidcounty                     0
regionidzip                       23
roomcnt                            0
yearbuilt                        112
structuretaxvaluedollarcnt        82
taxvaluedollarcnt                  1
assessmentyear                     0
landtaxvaluedollarcnt              1
taxamount                          4
censustractandblock              119
logerror                           0
transactiondate                    0
p

### 2. Rename Columns

In [10]:
df.rename(columns = {'bathroomcnt':'bathrooms', 'bedroomcnt':'bedrooms',
                              'calculatedfinishedsquarefeet':'total_sqft', 'finishedsquarefeet12': 'living_sqft', 'fullbathcnt':'full_bath', 'lotsizesquarefeet':'lot_sqft', 'regionidcity':'city', 'regionidzip':'zips', 'structuretaxvaluedollarcnt': 'structure_value', 'taxvaluedollarcnt':'assessed_value', 'landtaxvaluedollarcnt':'land_value'}, inplace = True)

In [16]:
df.columns

Index(['parcelid', 'bathrooms', 'bedrooms', 'calculatedbathnbr', 'total_sqft',
       'living_sqft', 'fips', 'full_bath', 'latitude', 'longitude', 'lot_sqft',
       'propertycountylandusecode', 'propertylandusetypeid',
       'rawcensustractandblock', 'city', 'regionidcounty', 'zips', 'roomcnt',
       'yearbuilt', 'structure_value', 'assessed_value', 'assessmentyear',
       'land_value', 'taxamount', 'censustractandblock', 'logerror',
       'transactiondate', 'propertylandusedesc', 'age', 'county'],
      dtype='object')

### 3. Data Mapping, Dummy Variables

In [12]:
df['age'] = 2017 - df['yearbuilt']

In [15]:
df['county'] = df.fips.map({6037: 'Los Angeles', 6059: 'Orange', 6111: 'Ventura'})

In [ ]:
df.latitude = df.latitude/1000000

In [ ]:
df.longitude = df.longitude/1000000

### 4. Drop Unuseful Columns

In [17]:
col = ['parcelid', 'yearbuilt','fips','propertycountylandusecode', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcounty', 'censustractandblock', 'propertylandusedesc']
df.drop(columns = col, inplace = True)

### 5. Data Types

In [23]:
df.dtypes

bathrooms            float64
bedrooms             float64
calculatedbathnbr    float64
total_sqft           float64
living_sqft          float64
full_bath            float64
latitude             float64
longitude            float64
lot_sqft             float64
city                 float64
zips                 float64
roomcnt              float64
structure_value      float64
assessed_value       float64
assessmentyear       float64
land_value           float64
taxamount            float64
logerror             float64
transactiondate       object
age                  float64
county                object
dtype: object

In [25]:
df['city'] = df['city'].astype(object)

In [ ]:
def split(df):
    '''
    This function drops the customer_id column and then splits a dataframe into 
    train, validate, and test in order to explore the data and to create and validate models. 
    It takes in a dataframe and contains an integer for setting a seed for replication. 
    Test is 20% of the original dataset. The remaining 80% of the dataset is 
    divided between valiidate and train, with validate being .30*.80= 24% of 
    the original dataset, and train being .70*.80= 56% of the original dataset. 
    The function returns, train, validate and test dataframes. 
    '''
    train, test = train_test_split(df, test_size = .2, random_state=123)   
    train, validate = train_test_split(train, test_size=.3, random_state=123)
    
    return train, validate, test

In [35]:
df.to_csv('zillow.csv')